In [1]:
!pip install -U langgraph langchain langchain-core langchain-community langchain-groq python-dotenv


In [3]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [ ]:
import os
from typing import TypedDict

from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, END


# Load environment variables

load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


# Initialize Groq LLM with UPDATED MODEL

llm = ChatGroq(
    model="llama-3.3-70b-versatile",  # ✅ Updated to supported model
    temperature=0
)


# Define State

class GraphState(TypedDict):
    messages: list


# Define Node

def chatbot_node(state: GraphState):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {
        "messages": messages + [response]
    }


# Build Graph

builder = StateGraph(GraphState)

builder.add_node("chatbot", chatbot_node)
builder.set_entry_point("chatbot")
builder.add_edge("chatbot", END)

graph = builder.compile()


# Run Graph - Example 1: Simple Question

print("=" * 60)
print("EXAMPLE 1: Simple Question")
print("=" * 60)

result = graph.invoke({
    "messages": [HumanMessage(content="Explain LangGraph in one sentence")]
})

print("\nResponse:")
print(result["messages"][-1].content)


# Run Graph - Example 2: Multi-turn Conversation

print("\n" + "=" * 60)
print("EXAMPLE 2: Multi-turn Conversation")
print("=" * 60)

result = graph.invoke({
    "messages": [
        HumanMessage(content="What is Python?"),
        AIMessage(content="Python is a high-level programming language."),
        HumanMessage(content="What are its main features?")
    ]
})

print("\nConversation:")
for msg in result["messages"]:
    role = "Human" if isinstance(msg, HumanMessage) else "AI"
    print(f"{role}: {msg.content}")


# Run Graph - Example 3: Code Generation

print("\n" + "=" * 60)
print("EXAMPLE 3: Code Generation")
print("=" * 60)

result = graph.invoke({
    "messages": [HumanMessage(content="Write a Python function to calculate factorial")]
})

print("\nResponse:")
print(result["messages"][-1].content)

EXAMPLE 1: Simple Question

Response:
LangGraph is an open-source, multilingual large language model developed by Llama that uses a graph-based architecture to process and generate human-like language, allowing for more efficient and flexible natural language understanding and generation capabilities.

EXAMPLE 2: Multi-turn Conversation

Conversation:
Human: What is Python?
AI: Python is a high-level programming language.
Human: What are its main features?
AI: **Main Features of Python:**

1. **Easy to Learn**: Python has a simple syntax and is relatively easy to learn, making it a great language for beginners.
2. **High-Level Language**: Python is a high-level language, meaning it abstracts away many low-level details, allowing developers to focus on the logic of their program.
3. **Interpreted Language**: Python code is interpreted line by line, making it easier to write and test code.
4. **Object-Oriented**: Python supports object-oriented programming (OOP) concepts, such as classes